In [2]:
from myutils import io
evalset = io.jsonload("data/eval.json")

gpt3.5、gpt4

In [ ]:
from myutils import net, gpt
net.set_proxy()
api_keys = []
result = []
i = 0
for unit in evalset:
    question = unit['question']
    lock = True
    while lock:
        try:
            api_key = api_keys[i]
            model_response = gpt.call_gpt(question, api_key, 'gpt-3.5-turbo')
            lock = False
        except Exception as e:
            print(e)
            i = (i+1)%len(api_keys)
    result.append({
        "question": question,
        "answer": model_response
    })
    io.jsondump(result, "evaluated_models/gpt3.5.json")
    i = (i+1)%len(api_keys)
net.unset_proxy()

chatglm-6b、chatglm2-6b、qwen7b、baichuan-13b